In [6]:
from Bio import SeqIO
import pandas as pd

Функция достаёт из генбанк-файла координаты mat_peptide, то есть белков в последовательности

In [7]:
def extract_mat_peptides(genbank_file):
    records = SeqIO.parse(genbank_file, "genbank")
    mat_peptides = []

    for record in records:
        for feature in record.features:
            if feature.type == "mat_peptide":
                location = feature.location
                product = feature.qualifiers.get('product', [''])[0]
                mat_peptides.append({
                    'Start': location.start+1,
                    'End': location.end,
                    'Product': product
                })
    protein_coordinates = pd.DataFrame(mat_peptides)
    return protein_coordinates

In [8]:
genbank_file = "../Records/reference_sequence.gb"
protein_coordinates = extract_mat_peptides(genbank_file)
protein_coordinates.to_csv('../Proteins/protein_coords.csv', index=False)

In [9]:
protein_coordinates

,Start,End,Product
0,1,84,p20a
1,85,612,p16
2,613,858,VP4
3,859,1512,VP2
4,1513,2175,VP3
5,2176,2811,VP1
6,2812,4734,p52
7,4735,4803,putative VPg1
8,4804,4875,putative VPg2
9,4876,4947,putative VPg3


Функция достаёт первую последовательность из fasta файла с выравниванием, так как в данном случае первая последовательность является референсной

In [10]:
def get_first_sequence(fasta_file):
    with open(fasta_file, "r") as handle:
        for record in SeqIO.parse(handle, "fasta"):
            return str(record.seq)

In [11]:
fasta_file = "../filtered_CDS_alignment.fasta"
reference_sequence = get_first_sequence(fasta_file)

Функция, по выравненной последовательности и по координатам белков в референсной последовательности, находит координаты этих белков в выравненной последовательности

In [12]:
def find_gapped_positions(sequence, ungapped_coordinates):
    gapped_coordinates = []
    gapped_index = 0
    for i, letter in enumerate(sequence):
        if letter != "-":
            gapped_index += 1
            if gapped_index in ungapped_coordinates:
                gapped_coordinates.append(i + 1)

    seq = sequence[gapped_coordinates[0]-1:gapped_coordinates[1]]            
    return seq, gapped_coordinates

Функция преобразует результаты работы предыдущей функции в таблицу

In [13]:
def process_table(table, sequence):
    results = []
    for index, row in table.iterrows():
        start = row['Start']
        end = row['End']
        product = row['Product']
        ungapped_coordinates = list((start, end))
        seq, gapped_coordinates = find_gapped_positions(sequence, ungapped_coordinates)
        results.append({
            'Product': product,
            'Sequence': seq,
            'Gapped_Coordinates': gapped_coordinates
        })
    results = pd.DataFrame(results)
    return results

In [14]:
results = process_table(protein_coordinates, reference_sequence)

In [15]:
results.to_csv('../Proteins/gapped_protein_coords.csv', index=False)